In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone

Dataset URL: https://www.kaggle.com/datasets/nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone
License(s): copyright-authors
ct-kidney-dataset-normal-cyst-tumor-and-stone.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/ct-kidney-dataset-normal-cyst-tumor-and-stone.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
!pip install split-folders

In [7]:
import splitfolders
splitfolders.ratio(
    "/content/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
   output="./dataset",
   seed=7,
   ratio=(0.7,0.15, 0.15)
)

Copying files: 12446 files [00:12, 964.69 files/s] 


In [8]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten,BatchNormalization,Dropout,Conv2D,MaxPooling2D
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import ConvNeXtLarge
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/dataset/train',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/content/dataset/test',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 8710 images belonging to 4 classes.
Found 1871 images belonging to 4 classes.


In [10]:
base_model = tensorflow.keras.applications.InceptionResNetV2(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg',
    classifier_activation='softmax',

)

In [35]:
base_model.trainable = True

# Flag to control when to start fine-tuning
set_trainable = False

# Unfreeze layers from a specific layer onwards
for layer in base_model.layers:
    if layer.name == 'block8_6_conv':  # Replace with the layer name from where you want to start fine-tuning
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Print the trainable status of each layer
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_layer False
conv2d False
batch_normalization False
activation False
conv2d_1 False
batch_normalization_1 False
activation_1 False
conv2d_2 False
batch_normalization_2 False
activation_2 False
max_pooling2d False
conv2d_3 False
batch_normalization_3 False
activation_3 False
conv2d_4 False
batch_normalization_4 False
activation_4 False
max_pooling2d_1 False
conv2d_8 False
batch_normalization_8 False
activation_8 False
conv2d_6 False
conv2d_9 False
batch_normalization_6 False
batch_normalization_9 False
activation_6 False
activation_9 False
average_pooling2d False
conv2d_5 False
conv2d_7 False
conv2d_10 False
conv2d_11 False
batch_normalization_5 False
batch_normalization_7 False
batch_normalization_10 False
batch_normalization_11 False
activation_5 False
activation_7 False
activation_10 False
activation_11 False
mixed_5b False
conv2d_15 False
batch_normalization_15 False
activation_15 False
conv2d_13 False
conv2d_16 False
batch_normalization_13 False
batch_normalization_16 False
ac

In [36]:
model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(124,activation='relu'))
model.add(Dense(124,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [37]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_resnet_v2 (Functional)     │ (None, 1536)                │      54,336,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 54,336,736 (207.28 MB)

 Trainable params: 12,268,576 (46.80 MB)

 Non-trainable params: 42,068,160 (160.48 MB)

In [39]:
history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

Epoch 1/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 305s 950ms/step - accuracy: 0.7888 - loss: 0.5643 - val_accuracy: 0.9417 - val_loss: 0.2058
Epoch 2/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 237s 850ms/step - accuracy: 0.9555 - loss: 0.1394 - val_accuracy: 0.9727 - val_loss: 0.0725
Epoch 3/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 236s 849ms/step - accuracy: 0.9804 - loss: 0.0655 - val_accuracy: 0.9369 - val_loss: 0.2220
Epoch 4/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 238s 856ms/step - accuracy: 0.9731 - loss: 0.0975 - val_accuracy: 0.9904 - val_loss: 0.0371
Epoch 5/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 238s 852ms/step - accuracy: 0.9861 - loss: 0.0473 - val_accuracy: 0.9503 - val_loss: 0.1574
Epoch 6/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 235s 844ms/step - accuracy: 0.9864 - loss: 0.0424 - val_accuracy: 0.9909 - val_loss: 0.0326
Epoch 7/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 236s 848ms/step - accuracy: 0.9906 - loss: 0.0360 - val_accuracy: 0.9947 - val_loss: 0.0192
Epoch 8/10
273/273 ━━━━━━━━━━━━━━━━━━━━ 263s 856ms/step - accuracy: 0.9912 -

In [ ]:


model.save('kidney_model2.h5')
